# Phase 1: Dataset Wrangling and Describing

In Phase 1, we load the main dataset, join it with additional data, and clean it.

The main dataset, [`FirstGenPokemon.csv`](https://www.kaggle.com/datasets/dizzypanda/gen-1-pokemon), comes from Kaggle. Meanwhile, additional data was retrieved from the Smogon website's [stats directory](https://www.smogon.com/stats/) (specifically, Gen 1 OU stats from [January 2016](https://www.smogon.com/stats/2016-01/gen1ou-0.txt) and [January 2025](https://www.smogon.com/stats/2025-01/gen1ou-0.txt)). Smogon is a community-run project dedicated to the competitive Pokemon scene and is a comprehensive and renowned resource for such.


In [10]:
# * Import necessary modules
import pandas as pd
import numpy as np

## First Gen Pokemon Dataset

In [ ]:
# * Load main data
main_df = pd.read_csv(r"dataset/FirstGenPokemon.csv")
 
 # ! Remove trailing space in column names
main_df.rename(columns=lambda label: label.strip(' '), inplace=True)

# Some Pokemon have no Type2, so we fill those with 'none'
main_df.fillna({'Type2': 'none'}, inplace=True)

# Set more appropriate datatypes
for col in ['Name', 'Type1', 'Type2']:
    main_df[col] = main_df[col].astype('string')

main_df.info()
main_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Number       151 non-null    int64  
 1   Name         151 non-null    string 
 2   Types        151 non-null    int64  
 3   Type1        151 non-null    string 
 4   Type2        151 non-null    string 
 5   Height(m)    151 non-null    float64
 6   Weight(kg)   151 non-null    float64
 7   Male_Pct     151 non-null    float64
 8   Female_Pct   151 non-null    float64
 9   Capt_Rate    151 non-null    int64  
 10  Exp_Points   151 non-null    int64  
 11  Exp_Speed    151 non-null    object 
 12  Base_Total   151 non-null    int64  
 13  HP           151 non-null    int64  
 14  Attack       151 non-null    int64  
 15  Defense      151 non-null    int64  
 16  Special      151 non-null    int64  
 17  Speed        151 non-null    int64  
 18  Normal_Dmg   151 non-null    float64
 19  Fire_Dmg

,Number,Name,Types,Type1,Type2,Height(m),Weight(kg),Male_Pct,Female_Pct,Capt_Rate,...,Poison_Dmg,Ground_Dmg,Flying_Dmg,Psychic_Dmg,Bug_Dmg,Rock_Dmg,Ghost_Dmg,Dragon_Dmg,Evolutions,Legendary
0,1,Bulbasaur,2,grass,poison,0.7,6.9,87.5,12.5,45,...,1.0,1.0,2.0,2.0,4.0,1.0,1,1,2,0
1,2,Ivysaur,2,grass,poison,1.0,13.0,87.5,12.5,45,...,1.0,1.0,2.0,2.0,4.0,1.0,1,1,2,0
2,3,Venusaur,2,grass,poison,2.0,100.0,87.5,12.5,45,...,1.0,1.0,2.0,2.0,4.0,1.0,1,1,2,0


## Data cleaning

Not all columns here are significantly related to competitive Pokemon, and some
of them can be extrapolated. These include:
* `Number` (Pokedex # that uniquely identifies a Pokemon)
* `Types` (Number of Types)
* `Capt_Rate` (Capture rate)
* `Height(m)`
* `Weight(kg)`: note that only 1 move, Low Kick, considers the weight of a Pokemon in Gen 1
* `Male_Pct` and `Female_Pct`: chance that a Pokemon is Male or Female (see footnote later)
* `Exp_Points` and `Exp_Speed`: experience-related stats
* `Evolutions`: number of evolutions this Pokemon has
* `Legendary`: if a Pokemon is considered a Legendary Pokemon

**However, before discarding these columns, note that there are actually two
rows listed for the Pokemon Nidoran:**

In [12]:
main_df[main_df['Name'] == 'Nidoran']

,Number,Name,Types,Type1,Type2,Height(m),Weight(kg),Male_Pct,Female_Pct,Capt_Rate,...,Poison_Dmg,Ground_Dmg,Flying_Dmg,Psychic_Dmg,Bug_Dmg,Rock_Dmg,Ghost_Dmg,Dragon_Dmg,Evolutions,Legendary
28,29,Nidoran,1,poison,none,0.4,7.0,0.0,100.0,235,...,0.5,2.0,1.0,2.0,2.0,1.0,1,1,2,0
31,32,Nidoran,1,poison,none,0.5,9.0,100.0,0.0,235,...,0.5,2.0,1.0,2.0,2.0,1.0,1,1,2,0


![A Pokedex listing starting from Nidoran](assets/two-nidorans.png)

fig. 1: Nidoran is listed twice in the Gen 1 Pokedex.

This is not a mistake. In the first generation of Pokemon games (hereafter referred to as _Gen 1_), Nidoran is listed twice due to stark gender differences in physical appearance. (Later generations of Pokemon would introduce gender as a mechanic.)

We therefore rename these two Pokemon into *NidoranM* and *NidoranF*, selecting them based off their Number:

In [13]:
main_df.loc[main_df['Number'] == 29, 'Name'] = 'NidoranM'
main_df.loc[main_df['Number'] == 32, 'Name'] = 'NidoranF'

main_df[main_df['Number'].isin([29, 32])]['Name']

28    NidoranM
31    NidoranF
Name: Name, dtype: string

Aside from gender differences, we specifically renamed them *NidoranM* and
*NidoranF* because the Smogon dataset already named them that way. We further
align the Kaggle names with the Smogon names (which align more closely to the
official names):

In [14]:
main_df.loc[main_df['Number'] == 83, 'Name'] = "Farfetch'd" # from "Farfetchd"
main_df.loc[main_df['Number'] == 122, 'Name'] = 'Mr.Mime' # from "MrMime"

We can now continue on with discarding the columns mentioned previously:

In [15]:
cleaned_main_df = main_df.drop(
    columns=[
        "Number",
        "Types",
        "Capt_Rate",
        "Height(m)",
        "Weight(kg)",  # only 1 move, Low Kick, takes weight into account in Gen1
        "Male_Pct",
        "Female_Pct",
        "Exp_Points",
        "Exp_Speed",
        "Evolutions",
        "Legendary",
    ]
)

cleaned_main_df

,Name,Type1,Type2,Base_Total,HP,Attack,Defense,Special,Speed,Normal_Dmg,...,Ice_Dmg,Fight_Dmg,Poison_Dmg,Ground_Dmg,Flying_Dmg,Psychic_Dmg,Bug_Dmg,Rock_Dmg,Ghost_Dmg,Dragon_Dmg
0,Bulbasaur,grass,poison,253,45,49,49,65,45,1.0,...,2.0,0.5,1.0,1.0,2.0,2.0,4.0,1.0,1,1
1,Ivysaur,grass,poison,325,60,62,63,80,60,1.0,...,2.0,0.5,1.0,1.0,2.0,2.0,4.0,1.0,1,1
2,Venusaur,grass,poison,425,80,82,83,100,80,1.0,...,2.0,0.5,1.0,1.0,2.0,2.0,4.0,1.0,1,1
3,Charmander,fire,none,249,39,52,43,50,65,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1,1
4,Charmeleon,fire,none,325,58,64,58,65,80,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Dratini,dragon,none,250,41,64,45,50,50,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,2
147,Dragonair,dragon,none,350,61,84,65,70,70,1.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,2
148,Dragonite,dragon,flying,500,91,134,95,100,80,1.0,...,4.0,0.5,1.0,0.0,1.0,1.0,0.5,2.0,1,2
149,Mewtwo,psychic,none,590,106,110,90,154,130,1.0,...,1.0,0.5,1.0,1.0,1.0,0.5,2.0,1.0,0,1


### Gender in Gen 1 Pokemon?

Gender as a game mechanic was introduced in the second generation of Pokemon
games (Gen 2), with Nidoran being the only Pokemon in Gen 1 whose gender can be
seen, which arouses suspicion about the `Male_Pct` and `Female_Pct` columns.
However, it turns out that Gen 1 Pokemon can be traded to a Gen 2 game, which
in turn "reveals" the gender of the said Pokemon.

## Smogon Datasets

Now that the Kaggle dataset is ready for merging, we now turn our attention to
the Smogon datasets.

These datasets were stored in the Smogon website in a plaintext format, so we
first created a [program](smogon-to-csv.py) to translate it into a `.csv` file.
(The specific commands used to run the program are in
[a batch file](run-smogon-to-csv.bat)).

We then use pandas to read the generated .csv files into a DataFrame and process
them further:

In [16]:
# * Load additional data
gen1_2016_df = pd.read_csv(r"dataset/2016-01-gen1ou-0.csv")
gen1_2025_df = pd.read_csv(r"dataset/2025-01-gen1ou-0.csv")

# * Rename '%' and '%.1' columns and set % cols to floats
for df in [gen1_2016_df, gen1_2025_df]:
    df.rename(columns={'%': 'Raw%', '%.1': 'Real%'}, inplace=True)
    for col in ['Usage%', 'Raw%', 'Real%']:
        df[col] = df[col].str.rstrip('%').astype(float) / 100.0

# * Tag all but the 'Pokemon' column in our additional data by year
# ! We rename the 'Pokemon' column to 'Name' to sync it with the main data
gen1_2016_df.rename(columns=lambda label: f"{label} (2016)" if label != 'Pokemon' else 'Name', inplace=True)
gen1_2025_df.rename(columns=lambda label: f"{label} (2025)" if label != 'Pokemon' else 'Name', inplace=True)

gen1_2016_df.info()
gen1_2025_df.info()
gen1_2025_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank (2016)    130 non-null    int64  
 1   Name           130 non-null    object 
 2   Usage% (2016)  130 non-null    float64
 3   Raw (2016)     130 non-null    int64  
 4   Raw% (2016)    130 non-null    float64
 5   Real (2016)    130 non-null    int64  
 6   Real% (2016)   130 non-null    float64
dtypes: float64(3), int64(3), object(1)
memory usage: 7.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank (2025)    136 non-null    int64  
 1   Name           136 non-null    object 
 2   Usage% (2025)  136 non-null    float64
 3   Raw (2025)     136 non-null    int64  
 4   Raw% (2025)    136 non-null    float64
 5   Real (2025)    136 no

,Rank (2025),Name,Usage% (2025),Raw (2025),Raw% (2025),Real (2025),Real% (2025)
0,1,Tauros,0.813940,38934,0.81394,29783,0.72327
1,2,Snorlax,0.719196,34402,0.71920,29453,0.71525
2,3,Chansey,0.679308,32494,0.67931,28807,0.69956


## Joining the datasets

With both the Kaggle and Smogon datasets ready, we merge them on their `Name`
column.

In [17]:
# Join data by Pokemon name
joined_df = cleaned_main_df.merge(gen1_2016_df, on='Name', how='outer').merge(gen1_2025_df, on='Name', how='outer')
joined_df.head(5)

,Name,Type1,Type2,Base_Total,HP,Attack,Defense,Special,Speed,Normal_Dmg,...,Raw (2016),Raw% (2016),Real (2016),Real% (2016),Rank (2025),Usage% (2025),Raw (2025),Raw% (2025),Real (2025),Real% (2025)
0,Abra,psychic,none,255,25,20,15,105,90,1.0,...,7.0,0.00059,7.0,0.00074,125.0,0.000042,2.0,0.00004,2.0,0.00005
1,Aerodactyl,rock,flying,440,80,105,65,60,130,0.5,...,180.0,0.01507,134.0,0.01410,31.0,0.015366,735.0,0.01537,605.0,0.01469
2,Alakazam,psychic,none,405,55,50,45,135,120,1.0,...,6390.0,0.53491,5475.0,0.57607,6.0,0.380252,18189.0,0.38025,16057.0,0.38994
3,Arbok,poison,none,359,60,85,69,65,80,1.0,...,56.0,0.00469,43.0,0.00452,65.0,0.003888,186.0,0.00389,147.0,0.00357
4,Arcanine,fire,none,455,90,110,80,80,95,1.0,...,629.0,0.05265,466.0,0.04903,35.0,0.012711,608.0,0.01271,493.0,0.01197


## Filling in post-merge NaNs

Some Pokemon did not appear in either the 2016 or 2025 Smogon data, because they
weren't used as often as the others. Because of this, we filled in their NaN
values with 0.0, to indicate "no usage".

In [18]:
# Clean NaNs.
cleaned_df = joined_df.fillna(0.0)
cleaned_df

,Name,Type1,Type2,Base_Total,HP,Attack,Defense,Special,Speed,Normal_Dmg,...,Raw (2016),Raw% (2016),Real (2016),Real% (2016),Rank (2025),Usage% (2025),Raw (2025),Raw% (2025),Real (2025),Real% (2025)
0,Abra,psychic,none,255,25,20,15,105,90,1.0,...,7.0,0.00059,7.0,0.00074,125.0,0.000042,2.0,0.00004,2.0,0.00005
1,Aerodactyl,rock,flying,440,80,105,65,60,130,0.5,...,180.0,0.01507,134.0,0.01410,31.0,0.015366,735.0,0.01537,605.0,0.01469
2,Alakazam,psychic,none,405,55,50,45,135,120,1.0,...,6390.0,0.53491,5475.0,0.57607,6.0,0.380252,18189.0,0.38025,16057.0,0.38994
3,Arbok,poison,none,359,60,85,69,65,80,1.0,...,56.0,0.00469,43.0,0.00452,65.0,0.003888,186.0,0.00389,147.0,0.00357
4,Arcanine,fire,none,455,90,110,80,80,95,1.0,...,629.0,0.05265,466.0,0.04903,35.0,0.012711,608.0,0.01271,493.0,0.01197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Weepinbell,grass,poison,345,65,90,50,85,55,1.0,...,0.0,0.00000,0.0,0.00000,77.0,0.002070,99.0,0.00207,65.0,0.00158
147,Weezing,poison,none,420,65,90,120,85,60,1.0,...,64.0,0.00536,54.0,0.00568,58.0,0.005519,264.0,0.00552,219.0,0.00532
148,Wigglytuff,normal,none,350,140,70,45,50,45,1.0,...,23.0,0.00193,16.0,0.00168,78.0,0.002007,96.0,0.00201,79.0,0.00192
149,Zapdos,electric,flying,490,90,90,85,125,100,1.0,...,2165.0,0.18123,1678.0,0.17656,9.0,0.245474,11742.0,0.24547,10278.0,0.24960
